# Install requirements

In [ ]:
!pip install fiftyone

In [ ]:
!fiftyone plugins download https://github.com/harpreetsahota204/synthetic_gui_samples_plugins.git --overwrite

# Download training dataset

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "harpreetsahota/FiftyOne-GUI-Grounding-Train",
    overwrite=True
    )

# Split datatset upfront

In [ ]:
import fiftyone.utils.random as four

four.random_split(dataset, {"train": 0.5, "val": 0.5})

train_view = dataset.match_tags("train").clone()

val_view = dataset.match_tags("val")

In [ ]:
train_view

# Using only training split to generate synthetic samples

Before running the plugins, open the terminal and launch a delegated service: `fiftyone delegated launch`

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo

rephraser = foo.get_operator("@harpreetsahota/synthetic_gui_samples_plugins/task_description_augment")

rephraser(
    train_view,
    model_name="Qwen/Qwen3-8B",
    mode="rephrase", #or use translate and pass in target_language
    target_language="",
    enable_thinking=False, # you can enable thinking for better results but will take much longer
    process_keypoints=True,
    process_detections=True,
    delegate=True
    )

In [ ]:
import time

def keep_running():
  """Keeps the cell running indefinitely."""
  while True:
    print("Cell is still running...")
    time.sleep(6000000) # Sleep for 60 seconds

keep_running()

Pick one, or more, of the following. Depends on your disk space.

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo

colorblind_sim_augment = foo.get_operator("@harpreetsahota/synthetic_gui_samples_plugins/colorblind_sim_augment")

colorblind_sim_augment(
    train_view,
    colorblind_type="tritanopia",
    copy_keypoints=True,
    copy_detections=True,
    delegate=True
    )

In [ ]:
import fiftyone as fo
import fiftyone.operators as foo

invert_colors_augment = foo.get_operator("@harpreetsahota/synthetic_gui_samples_plugins/invert_colors_augment")

invert_colors_augment(
    train_view,
    copy_keypoints=True,
    copy_detections=True,
    delegate=True
    )

In [ ]:
resizer = foo.get_operator("@harpreetsahota/synthetic_gui_samples_plugins/resize_images")

resizer(
    train_view,
    use_custom=True,
    target_width=1024,
    target_height=768,
    copy_keypoints=True,
    copy_detections=True,
    delegate=True
    )

You can apply these other transforms if you'd like. However, it will take up significant amounts of disk space.

In [ ]:
# colorblind_sim_augment(
#     train_view,
#     colorblind_type="protanopia",
#     copy_keypoints=True,
#     copy_detections=True,
#     delegate=True
#     )

In [ ]:
# colorblind_sim_augment(
#     train_view,
#     colorblind_type="deuteranopia",
#     copy_keypoints=True,
#     copy_detections=True,
#     delegate=True
#     )

In [ ]:
# grayscale_augment = foo.get_operator("@harpreetsahota/synthetic_gui_samples_plugins/grayscale_augment")

# grayscale_augment(
#     train_view,
#     copy_keypoints=True,
#     copy_detections=True,
#     delegate=True
#     )

In [ ]:
# resizer(
#     train_view,
#     use_custom=False,
#     target_width=1440,
#     target_height=900,
#     copy_keypoints=True,
#     copy_detections=True,
#     delegate=True
#     )

To keep track of your runs, you can run `fiftyone delegated launch` in your termninal (or in your notebook, as shown below)

In [ ]:
!fiftyone delegated list

In [ ]:
train_view

Combine the training dataset (which now has synthtic samples) with the validation dataset:

In [ ]:
train_view.tag_samples("train")

train_view.save()

new_dataset = train_view.concat(dataset.match_tags("val").clone())

new_dataset.name = "FiftyOne-GUI-Grounding-Train-with-Synthetic"

And push it to the Hugging Face Hub:

In [ ]:
from fiftyone.utils.huggingface import push_to_hub

push_to_hub(
    new_dataset,
    "FiftyOne-GUI-Grounding-Train-with-Synthetic",
    exist_ok=True
)

You can test that the dataset was successfully uploaded as follows:

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

test_dataset = load_from_hub(
    "harpreetsahota/FiftyOne-GUI-Grounding-Train-with-Synthetic",
    overwrite=True
    )

In [ ]:
fo.launch_app(test_dataset)